In [ ]:
%matplotlib inline
# warnings.filterwarnings('ignore')

In [0]:
from fastai.vision import *
import ssl
import torch
import numpy as np
import torchvision.transforms as transforms
ssl._create_default_https_context = ssl._create_unverified_context
import zipfile
from IPython.display import Image

# Google Colab

This section is used to mount Drive directories if this notebook is run in Colab.

In [0]:
# Load the Drive helper and mount
from google.colab import drive, files

# This will prompt for authorization.
drive.mount('/content/drive')

In [0]:
!ls "/content/drive/Shared drives/Class 15"

In [0]:
# train set
path = Path("/content/drive/Shared drives/Class 15/train-v6") 

# # unzip file
# path1 = path/'train-v4.zip'

# with zipfile.ZipFile(path1, 'r') as zip_ref:
#     zip_ref.extractall(path)

In [0]:
# Fill 'other' folder
folder_other = 'other'
file_other = 'other.csv'

#path = Path('/content/drive/Shared drives/Class 15/train-v6')
dest = path/folder_other 
dest.mkdir(parents=True, exist_ok=True)
download_images(path/file_other, dest, max_pics=40, max_workers=0)

# Check if the images in path aren't broken, maybe resize them and copy it in dest.
verify_images(path/'other', delete=True, max_size=200)

# View data

In [0]:
classes = ['asc',
 'beterleven',
 'demeter',
 'ebio',
 'eko',
 'fairtrade',
 'msc',
 'other',
 'ralliance',
 'utz']

In [0]:
# for c in classes:
#     print(other)
#     verify_images(path/'other', delete=True, max_size=50)

In [0]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2, size=224, num_workers=4).normalize(imagenet_stats)

In [0]:
data.classes

In [0]:
data.show_batch(rows=3, figsize=(7,8))

In [0]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [0]:
data.to_df()['y'].value_counts()

## Data augmentation

In [0]:
path1 = Path("/content/drive/Shared drives/Class 15/train-v6") 

In [0]:
data_aug = ImageDataBunch.from_folder(path1, train = ".", valid_pct = 0.2, 
                         ds_tfms = get_transforms(do_flip=True, flip_vert=False, 
                         max_rotate=10.0, max_zoom=1.1, max_lighting=0.2, 
                         max_warp=0.2, p_affine=0.75, p_lighting=0.75, xtra_tfms=None), 
                         size=224, num_workers=4).normalize(imagenet_stats)

#data_aug.show_batch(rows=3, figsize=(5,5))

In [0]:
# create a train.txt file 
!touch "/content/drive/Shared drives/Class 15/train-v7"

## Train model

In [0]:
learn = cnn_learner(data_aug, models.resnet34, metrics=error_rate)

In [0]:
learn.fit_one_cycle(4)

In [0]:
learn.save('stage-1')
learn.unfreeze()
#learn.lr_find()

In [0]:
# find learning rate
learn.recorder.plot(suggestion = True)

In [0]:
learn.fit_one_cycle(12, max_lr=slice(1e-06, 1e-03))

In [0]:
learn.save('stage-2')

## Interpretation

In [0]:
#learn.load('stage-2')

In [0]:
interp = ClassificationInterpretation.from_learner(learn)

In [0]:
interp.plot_confusion_matrix()

In [0]:
interp.most_confused(min_val=1)

In [0]:
preds,y, loss = learn.get_preds(with_loss=True)

# get accuracy
acc = accuracy(preds, y)
print('The accuracy is {0} %.'.format(acc))

In [0]:
interp.plot_top_losses(9)

In [0]:
from sklearn.metrics import roc_curve, auc
# probs from log preds
probs = np.exp(preds[:,1])
# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y, probs, pos_label=1)

# Compute ROC area
roc_auc = auc(fpr, tpr)
print('ROC area is {0}'.format(roc_auc))

In [0]:
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")

## Putting your model in production

In [0]:
learn.export()

In [0]:
defaults.device = torch.device('cpu')

In [0]:
img1 = open_image(path1/'utz/IMG_2939.jpg')
img1

# pred_class,pred_idx,outputs = learn.predict(img)
pred_class = learn.predict(img1)
pred_class
# result: it predicted the image as a “utz” with ..% probs

In [0]:
img2 = open_image(path/'msc/IMG_8448.JPG')
img2

# pred_class,pred_idx,outputs = learn.predict(img)
pred_class = learn.predict(img2)
pred_class
# result: it predicted the image as a “msc” with ..% probs

In [0]:
img3 = open_image(path/'utz'/'00000001.png')
img3

# pred_class,pred_idx,outputs = learn.predict(img)
pred_class = learn.predict(img3)
pred_class
# result: it predicted the image as a “utz” with ..% probs

In [0]:
learn = load_learner(path)

## Deploy

In [0]:
# This will create an export.pkl file that you'll need to copy with your model file if you want to deploy it on another device.
# This saves the internal information (classes, etc) need for inference in a file named 'export.pkl'. 
learn.export('./export1.pkl')

In [0]:
######## RUN THIS ON A NEW MACHINE ##########
from fastai.vision import * # fastai 1.0
from fastai import *
from torchvision.models import *

arch = resnet34       # specify model architecture
MODEL_PATH = str(arch).split()[1] + '_stage2'
empty_data = ImageDataBunch.load_empty('./') #this will look for a file named export.pkl in the specified path
learner = create_cnn(empty_data, arch).load(MODEL_PATH)

In [0]:
classes = data.classes
classes

In [0]:
path1 = Path("/content/drive/Shared drives/Class 15/train-v6") 
img = open_image(path1/'zon.png')

## predict and visualize
img.show(y=learn.predict(img)[0])
print('This is a ' + classes[int(learn.predict(img)[0])] + ' quality mark.')

In [0]:
model = pickle.load(open(Path("/content/drive/Shared drives/Class 15/train-v6/export1.pkl"), 'rb'))

In [0]:
model